# Ollama + OpenAI + Python

## 1. Spécifiez le nom du modèle

Si vous avez utilisé un modèle différent de "phi3:mini", modifiez la valeur dans la cellule ci-dessous. Cette variable sera utilisée dans le code tout au long du notebook.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Configurer le client Open AI

Le client OpenAI est généralement utilisé avec OpenAI.com ou Azure OpenAI pour interagir avec des modèles de langage avancés.  
Cependant, il peut également être utilisé avec Ollama, car Ollama offre un point de terminaison compatible avec OpenAI à "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Générer une réponse de chat

Nous pouvons maintenant utiliser le SDK OpenAI pour générer une réponse à une conversation. Cette requête devrait produire un haïku sur les chats :


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Ingénierie des invites

Le premier message envoyé au modèle de langage est appelé le "message système" ou "invite système", et il définit les instructions générales pour le modèle.  
Vous pouvez fournir votre propre invite système personnalisée pour orienter un modèle de langage afin qu'il génère des réponses d'une manière différente.  
Modifiez le `SYSTEM_MESSAGE` ci-dessous pour répondre comme votre personnage préféré de film ou série télévisée, ou trouvez de l'inspiration pour d'autres invites système sur [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Une fois que vous avez personnalisé le message système, posez la première question utilisateur dans le `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Quelques exemples

Une autre manière de guider un modèle de langage est de fournir des "quelques exemples", une série de questions/réponses qui montrent comment il devrait répondre.

L'exemple ci-dessous tente de faire agir un modèle de langage comme un assistant pédagogique en fournissant quelques exemples de questions et réponses qu'un assistant pourrait donner, puis en posant au modèle une question qu'un étudiant pourrait poser.

Essayez d'abord, puis modifiez le `SYSTEM_MESSAGE`, les `EXAMPLES` et le `USER_MESSAGE` pour un nouveau scénario.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Génération Augmentée par la Recherche

RAG (Génération Augmentée par la Recherche) est une technique permettant à un modèle de langage de répondre avec précision à des questions spécifiques à un domaine, en récupérant d'abord des informations pertinentes à partir d'une source de connaissances, puis en générant une réponse basée sur ces informations.

Nous avons fourni un fichier CSV local contenant des données sur les voitures hybrides. Le code ci-dessous lit le fichier CSV, recherche des correspondances avec la question de l'utilisateur, puis génère une réponse basée sur les informations trouvées. Notez que cela prendra plus de temps que les exemples précédents, car davantage de données sont envoyées au modèle. Si vous remarquez que la réponse n'est toujours pas basée sur les données, vous pouvez essayer l'ingénierie du système ou tester d'autres modèles. En général, RAG est plus efficace avec des modèles plus grands ou avec des versions ajustées des SLMs.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
